# Imports

In [2]:
import os
import numpy as np
import open3d as o3d
import trimesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Functions

In [3]:
def highlight_holes(mesh):
    broken = trimesh.repair.broken_faces(mesh, color=[255,0,0,125])
    print(f"Number of Holes: {len(broken)}")

    return mesh

def edges_to_lineset(mesh, edges, color):
    ls = o3d.geometry.LineSet()
    ls.points = mesh.vertices
    ls.lines = edges
    ls.paint_uniform_color(color)
    return ls

def check_properties(mesh):
    mesh.compute_vertex_normals()

    edge_manifold = mesh.is_edge_manifold(allow_boundary_edges=True)
    edge_manifold_boundary = mesh.is_edge_manifold(allow_boundary_edges=False)
    vertex_manifold = mesh.is_vertex_manifold()
    self_intersecting = mesh.is_self_intersecting()
    watertight = mesh.is_watertight()
    orientable = mesh.is_orientable()

    print(f"  edge_manifold:          {edge_manifold}")
    print(f"  edge_manifold_boundary: {edge_manifold_boundary}")
    print(f"  vertex_manifold:        {vertex_manifold}")
    print(f"  self_intersecting:      {self_intersecting}")
    print(f"  watertight:             {watertight}")
    print(f"  orientable:             {orientable}")

    geoms = [mesh]
    if not edge_manifold:
        edges = mesh.get_non_manifold_edges(allow_boundary_edges=True)
        geoms.append(edges_to_lineset(mesh, edges, (1, 0, 0)))
    if not edge_manifold_boundary:
        edges = mesh.get_non_manifold_edges(allow_boundary_edges=False)
        geoms.append(edges_to_lineset(mesh, edges, (0, 1, 0)))
    if not vertex_manifold:
        verts = np.asarray(mesh.get_non_manifold_vertices())
        pcl = o3d.geometry.PointCloud(
            points=o3d.utility.Vector3dVector(np.asarray(mesh.vertices)[verts]))
        pcl.paint_uniform_color((0, 0, 1))
        geoms.append(pcl)
    if self_intersecting:
        intersecting_triangles = np.asarray(
            mesh.get_self_intersecting_triangles())
        intersecting_triangles = intersecting_triangles[0:1]
        intersecting_triangles = np.unique(intersecting_triangles)
        print("  # visualize self-intersecting triangles")
        triangles = np.asarray(mesh.triangles)[intersecting_triangles]
        edges = [
            np.vstack((triangles[:, i], triangles[:, j]))
            for i, j in [(0, 1), (1, 2), (2, 0)]
        ]
        edges = np.hstack(edges).T
        edges = o3d.utility.Vector2iVector(edges)
        geoms.append(edges_to_lineset(mesh, edges, (1, 1, 1)))
    o3d.visualization.draw_geometries(geoms, mesh_show_back_face=True, window_name="edge_manifold: R, edge_manifold_boundary: G, vertex_manifold: B, self_intersecting: White")

# Paths

In [4]:
original_meshes_path = "../../../../../vol/aimspace/users/wyo/organ_meshes_ply/"
dec_meshes_path = "../../../../../vol/aimspace/users/wyo/registered_meshes/2000/"
vc_meshes_path = "../../../../../vol/aimspace/users/wyo/organ_decimations_ply/vertex_clustering/"

# Data

In [3]:
dirs = next(os.walk(original_meshes_path))[1]
organs = ["liver_mesh.ply", "spleen_mesh.ply", "left_kidney_mesh.ply", "right_kidney_mesh.ply", "pancreas_mesh.ply"]

# Checking Water Tightness

In [4]:
water_tightness = {"liver": [0, 0, 0, 0, 0, 0], "left_kidney": [0, 0, 0, 0, 0, 0], "right_kidney": [0, 0, 0, 0, 0, 0], 
                    "spleen": [0, 0, 0, 0, 0, 0], "pancreas": [0, 0, 0, 0, 0, 0]}
output = {"True": 1, "False": 0}

mesh_path = f'{original_meshes_path}{str(dirs[0])}/{organs[0]}'
mesh = o3d.io.read_triangle_mesh(mesh_path)
output[str(mesh.is_watertight())]

0

In [7]:
for dir in dirs:
    for organ in organs:
        original_mesh_path = f'{original_meshes_path}{str(dir)}/{organ}'
        dec_mesh_path = f'{dec_meshes_path}{str(dir)}/{organ}'
        vc_mesh_path = f'{vc_meshes_path}{str(dir)}/{organ}'

        original_mesh = o3d.io.read_triangle_mesh(original_mesh_path)
        dec_mesh = o3d.io.read_triangle_mesh(dec_mesh_path)
        vc_mesh = o3d.io.read_triangle_mesh(vc_mesh_path)

        original_watertight = original_mesh.is_watertight()
        dec_watertight = dec_mesh.is_watertight()
        vc_watertight = vc_mesh.is_watertight()

        water_tightness[str(organ[:-9])][output[str(original_watertight)]] += 1
        water_tightness[str(organ[:-9])][output[str(dec_watertight)] + 2] += 1
        water_tightness[str(organ[:-9])][output[str(vc_watertight)] + 4] += 1

        print(original_watertight, dec_watertight, vc_watertight)
        print("Order: [original_not_wt, original_wt, dec_not_wt, dec_wt, vc_not_wt, vc_wt]")
        print(water_tightness)
        break
    break
    

False False False
Order: [original_not_wt, original_wt, dec_not_wt, dec_wt, vc_not_wt, vc_wt]
{'liver': [4, 0, 4, 0, 4, 0], 'left_kidney': [1, 0, 1, 0, 1, 0], 'right_kidney': [1, 0, 1, 0, 1, 0], 'spleen': [1, 0, 1, 0, 1, 0], 'pancreas': [1, 0, 1, 0, 1, 0]}


# Visualize Holes

In [5]:
mesh = o3d.io.read_triangle_mesh("../../local_data/original_meshes/2901448/liver_mesh.ply")
# mesh = o3d.io.read_triangle_mesh("../../local_data/decimations/female/1000071/liver_mesh.ply")
mesh.paint_uniform_color([0, 0, 0])
check_properties(mesh)
mesh

  edge_manifold:          True
  edge_manifold_boundary: True
  vertex_manifold:        True
  self_intersecting:      True
  watertight:             False
  orientable:             True
  # visualize self-intersecting triangles


TriangleMesh with 23338 points and 46672 triangles.

# Centering

In [22]:
mesh = o3d.io.read_triangle_mesh("../../local_data/registered_only/male/2901448/liver_mesh.ply")
mesh2 = o3d.io.read_triangle_mesh("../../local_data/registered_only/male/6022586/liver_mesh.ply")
pointSet = o3d.geometry.PointCloud()
pointSet.points = o3d.utility.Vector3dVector(np.zeros((1,3)))
mesh.paint_uniform_color((0, 1, 0))
mesh2.paint_uniform_color((0, 0, 1))
pointSet.paint_uniform_color((1, 0, 0))

center = mesh.get_center()
mesh.translate(-center, relative=True)
center = mesh2.get_center()
mesh2.translate(-center, relative=True)

TriangleMesh with 20608 points and 41218 triangles.

In [23]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(mesh)
vis.add_geometry(mesh2)
vis.add_geometry(pointSet)
vis.get_render_option().point_size = 20
vis.get_render_option().mesh_show_back_face = True
vis.get_render_option().mesh_show_wireframe = True
vis.run()
vis.destroy_window()